# 1. Import and Install Dependencies

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

# 2. Initial Function Definitions

In [ ]:
#Defining mediapipe holistic model and model to draw landmarks

mpHolistic = mp.solutions.holistic 
mpDraw = mp.solutions.drawing_utils
actions = np.array(['yes', 'no', 'i love you', 'happy'])


#Function to take in a frame as input, extract face, hand and pose landmarks and return it to the user

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

#Function that takes image and its landmarks as input and uses the mediapipe drawing model to draw landmarks

def draw_landmarks(image, results):
    mpDraw.draw_landmarks(image, results.pose_landmarks, mpHolistic.POSE_CONNECTIONS,
                             mpDraw.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mpDraw.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mpDraw.draw_landmarks(image, results.left_hand_landmarks, mpHolistic.HAND_CONNECTIONS, 
                             mpDraw.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mpDraw.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mpDraw.draw_landmarks(image, results.right_hand_landmarks, mpHolistic.HAND_CONNECTIONS, 
                             mpDraw.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mpDraw.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

#Function to extract the keypoints from the face, hand and pose landmarks and return the values in one array

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

#Function to visualize and render the probability of a particular action live on the OpenCV display interface

colors = (117,245,16)

def prob_viz(action, input_frame, colors):
    output_frame = input_frame.copy()
    cv2.rectangle(output_frame, (0,60), (90), colors, -1)
    cv2.putText(output_frame, action, (0, 85), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)        
    return output_frame

# 3. Setup Folders for Collection

In [ ]:
#Setting up storage spaces for storing each frame of data for every video for each action we are training the model on

try: 
    os.mkdir('MP_Data_new')
except:
    pass

DATA_PATH = os.path.join('MP_Data_new') 

# Actions that we try to detect
actions = np.array(['yes', 'no', 'i love you', 'happy'])

for action in actions:
    try: 
        os.makedirs(os.path.join(DATA_PATH, action))
    except:
        pass

numVids = 20

framesPerVid = 60

for action in actions: 
    for sequence in range(numVids):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 4. Collect Keypoint Values for Training and Testing

In [15]:
#Collection of data on the keypoints and landmark positions for each sign language action in order to train the model on it
numVids = 20
DATA_PATH = os.path.join('MP_Data_new') 
framesPerVid = 60
i = 3
cap = cv2.VideoCapture(0)
action = actions[i]

with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for sequence in range(numVids):
        for frameNum in range(framesPerVid):
            ret, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            draw_landmarks(image, results)

            if frameNum == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (150,200), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence + 1), (130,250), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(1000)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence + 1), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)

            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frameNum))
            np.save(npy_path, keypoints)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break              
cap.release()
cv2.destroyAllWindows() 
i+=1

# 5. Preprocess Data and Create Labels and Features

In [16]:
#Extracting the keypoint and landmark information for each action from its storage location and splitting all the data into respective training and testing datasets
actions = np.array(['yes', 'no', 'i love you', 'happy'])
DATA_PATH = os.path.join('MP_Data_new')
framesPerVid = 60
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []

for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frameNum in range(framesPerVid):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frameNum)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
# y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], y_train.shape[2])
# y_test = y_test.reshape(y_test.shape[0], y_test.shape[1], y_test.shape[2])


# 6. Build and Train LSTM Neural Network, and evaluate the model to determine accuracy

In [17]:
#Building the LSTM and Dense layers and training the model on the training dataset, as well as logging its live accuracy and loss percentage with each epoch

# log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir=log_dir)

# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,258)))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(actions.shape[0], activation='softmax'))
# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# # %tensorboard --logdir logs
# history = model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])
# model.summary()

# model.save('Assets/test.h5')

# ytrue = np.argmax(y_test, axis=1).tolist()
# ypred = np.argmax(model.predict(X_test), axis=1).tolist()

# print(multilabel_confusion_matrix(ytrue, ypred))
# print(accuracy_score(ytrue, ypred))

# from matplotlib import pyplot as plt

# plt.plot(history.history['categorical_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()

# plt.plot(history.history['loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()

In [18]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,258))))
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='relu')))
model.add(Bidirectional(LSTM(64, return_sequences=False, activation='relu')))
# model.add(Bidirectional(LSTM(units=64, input_shape=(X_train.shape[0], y_train.shape[0]))))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# %tensorboard --logdir logs
history = model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])
model.summary()

#model.save('Assets/test.h5')

ytrue = np.argmax(y_test, axis=1).tolist()
ypred = np.argmax(model.predict(X_test), axis=1).tolist()

print(multilabel_confusion_matrix(ytrue, ypred))
print(accuracy_score(ytrue, ypred))

from matplotlib import pyplot as plt
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# plt.plot(history.history['categorical_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['accuracy', 'val'], loc='upper left')
# plt.show()

# plt.plot(history.history['loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['loss', 'val'], loc='upper left')
# plt.show()

Epoch 1/500
3/3 [==============================] - 14s 94ms/step - loss: 2067.2253 - accuracy: 0.2941
Epoch 2/500
3/3 [==============================] - 0s 74ms/step - loss: 1845.7220 - accuracy: 0.2500
Epoch 3/500
3/3 [==============================] - 0s 76ms/step - loss: 1786.7045 - accuracy: 0.2647
Epoch 4/500
3/3 [==============================] - 0s 75ms/step - loss: 1.3865 - accuracy: 0.2500
Epoch 5/500
3/3 [==============================] - 0s 78ms/step - loss: 1.3865 - accuracy: 0.2500
Epoch 6/500
3/3 [==============================] - 0s 58ms/step - loss: 1.3865 - accuracy: 0.2500
Epoch 7/500
3/3 [==============================] - 0s 59ms/step - loss: 1.3864 - accuracy: 0.2500
Epoch 8/500
3/3 [==============================] - 0s 64ms/step - loss: 1.3866 - accuracy: 0.2500
Epoch 9/500
3/3 [==============================] - 0s 71ms/step - loss: 1.3865 - accuracy: 0.1765
Epoch 10/500
3/3 [==============================] - 0s 78ms/step - loss: 1.3865 - accuracy: 0.2500
Epoch 11/

KeyError: 'categorical_accuracy'

# 7. Applying model in real time to detect signs

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val'], loc='upper left')
# plt.ylim(0,3)
plt.show()

In [ ]:
model.save('Assets/signlangnew.h5')

In [ ]:
model = tf.keras.models.load_model('Assets/signlangnew.h5')
model.summary()

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
ypred = np.argmax(model.predict(X_test), axis=1).tolist()

print(multilabel_confusion_matrix(ytrue, ypred))
print(accuracy_score(ytrue, ypred))

In [ ]:
# Applying the model in real time to each frame of data recieved from the user in order to detect the signs being displayed
model = tf.keras.models.load_model('signlangnew.h5')
actions = np.array(['yes', 'no', 'i love you', 'happy', 'sad'])
sequence = []
full = ''
sentence = ''
prev_sen = ''
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        print(len(sequence))
        sequence = sequence[-60:]
        print(np.expand_dims(sequence, axis=0).shape)
        
        if len(sequence) == 60:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(res)
            predictions.append(np.argmax(res))
            # print(predictions)
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold:                     
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence:
                            sentence = actions[np.argmax(res)]
                    else:
                        sentence = actions[np.argmax(res)]

            if res[prediction] > threshold:                     
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence:
                        sentence = actions[np.argmax(res)]
                else:
                    sentence = actions[np.argmax(res)]
            image = prob_viz(actions[np.argmax(res)], image, colors)
            
        cv2.rectangle(image, (0,0), (640, 100), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(full), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)
        import os
        from gtts import gTTS

        if sentence != prev_sen:
            full += sentence + " "
            tts = gTTS(sentence)
            tts.save("Assets/output.mp3")
            os.system(f"say {sentence}")
            os.system("mpg321 Assets/output.mp3")  # On Linux
            os.system("afplay Assets/output.mp3")  # On macOS
            os.system("start Assets/output.mp3")
            prev_sen = sentence

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:

# import os

# # Text you want to convert to speech
# text = "Hello, this is a simple text-to-speech example in Python."

# # Initialize a gTTS object


# # Save the audio as an MP3 file
# tts.save("output.mp3")

# # Play the audio using the default system player
# os.system("mpg321 output.mp3")  # On Linux
# os.system("afplay output.mp3")  # On macOS
# os.system("start output.mp3")  # On Windows



In [ ]:
# cap.release()
# cv2.destroyAllWindows() 

In [ ]:
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))